## Word_embedding

Word embedding, bir kelime veya ifadenin düz bir vektör olarak ifade edilmesidir. Bu vektörler, kelimelerin arasındaki benzerlikleri yansıtmak üzere model tarfından öğrenilir. Bu sayede, yapay zeka modelleri kelimeler arasındaki ilişkileri daha iyi anlayabilir

Word embedding, metin verileri kullanılarak öğrenilir. Örneğin, bir metin corpus'u (büyük metin veritabanı) kullanılarak, her kelime için çevresindeki kelimeler ile ilişkisini öğrenir. Bu ilişki, kelime ve çevresindeki kelimeler arasındaki co-occurrence (ortak görünüm) frekansı olarak tanımlanır. Daha sonra, bu co-occurrence verileri kullanarak, her kelime için bir vektör oluşturulur. Bu vektörler, kelimelerin anlamları arasındaki benzerlikleri yansıtmak üzere tasarlanır.

Örneğin, "öğretmen", "öğrenci", "not verme" kelimeleri sıklıkla aynı metinlerde görülür ve benzer anlamlar taşır, bu nedenle oluşan vektörleri birbirine yakın olacak. Ancak "öğretmen" ve "portakal" kelimeleri sıklıkla aynı metinlerde görülmez ve farklı anlamlar taşır, bu nedenle oluşan vektörleri birbirinden uzak olacak.

## Word2Vec

In [ ]:
# pip install gensim
# conda install -c conda-forge gensim

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from nltk.tokenize import word_tokenize
import pandas as pd
from gensim.models import Word2Vec
import nltk

In [3]:
nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_csv('/content/drive/MyDrive/newspaper.zip', names = ["news"])
df

# pandas'ın read_csv fonksiyonu zip'li dosyaları da okuyabiliyor.
# ziplediğimiz data(corpus) txt dosyası olduğundan dosyada belirlenmiş feature ismi/isimleri yoktur. Bu sebeple
# feature isimlendirmesi için names parametresini kullanıyoruz.
# names parametresi kullanılmaz ise ilk satırdaki text default olarak feature names olarak atanır.

,news
0,iran devlet televizyonu ülkedeki eyaletin sind...
1,gösterilerde fitnecilere ölüm münafıklara ölüm...
2,dini lider ali hamaney ve cumhurbaşkanı mahmud...
3,musevi ye ölüm ve idam idam sloganları duyuldu
4,muhalefet liderleri kaçtı mı aşure günü yaşana...
...,...
411520,dışişleri bakanlığı ndan yapılan yazılı açıkla...
411521,açıklamada abd nin ankara büyükelçiliği ve ist...
411522,seyahat uyarısı güncelleme kararının temmuz da...
411523,amerikalı turistlerin açıkça türkiye deki ulus...


In [5]:
df.news[0]

'iran devlet televizyonu ülkedeki eyaletin sinde yapılan reformcuları protesto amaçlı yürüyüşlere milyonlarca kişinin katıldığını bildirdi '

In [6]:
word_tokenize(df.news[0])

['iran',
 'devlet',
 'televizyonu',
 'ülkedeki',
 'eyaletin',
 'sinde',
 'yapılan',
 'reformcuları',
 'protesto',
 'amaçlı',
 'yürüyüşlere',
 'milyonlarca',
 'kişinin',
 'katıldığını',
 'bildirdi']

In [7]:
%%time
corpus=[word_tokenize(i) for i in df.news]
print(corpus[:5])

# word2vec algoritması tüm corpusun 2 boyutlu olmasını ister. Bu sebeple burdaki for döngüsü ile tüm documnetleri/satırları
# teker teker çekip word tokenlerine ayırıyoruz. Word_tokenize fonksiyonu default olarak texti word tokenlerine ayırıp bir
# listeye atar.

[['iran', 'devlet', 'televizyonu', 'ülkedeki', 'eyaletin', 'sinde', 'yapılan', 'reformcuları', 'protesto', 'amaçlı', 'yürüyüşlere', 'milyonlarca', 'kişinin', 'katıldığını', 'bildirdi'], ['gösterilerde', 'fitnecilere', 'ölüm', 'münafıklara', 'ölüm', 'abd', 'ye', 'ölüm', 'ingiltere', 'ye', 'ölüm', 'sloganları', 'atıldı'], ['dini', 'lider', 'ali', 'hamaney', 've', 'cumhurbaşkanı', 'mahmud', 'ahmedinejad', 'ı', 'destekleyen', 'iranlılar', 'son', 'olaylarda', 'yeğeni', 'öldürülen', 'mir', 'hüseyin', 'musevi', 'başta', 'olmak', 'üzere', 'muhalefet', 'liderlerini', 'kınadılar'], ['musevi', 'ye', 'ölüm', 've', 'idam', 'idam', 'sloganları', 'duyuldu'], ['muhalefet', 'liderleri', 'kaçtı', 'mı', 'aşure', 'günü', 'yaşanan', 'çatışmalarda', 'devlet', 'kaynaklarına', 'göre', 'u', 'terörist', 'olmak', 'üzere', 'kişi', 'ölmüştü']]
CPU times: user 1min 7s, sys: 768 ms, total: 1min 8s
Wall time: 1min 9s


In [8]:
from gensim.utils import effective_n_jobs

effective_n_jobs(-1)

# bilgisayarınızdaki eğitim için kullanabileceğiniz max çekirdek sayısını gösterir.

2

In [9]:
%%time
model = Word2Vec(sentences=corpus,
                 alpha=0.025,
                 vector_size=100,
                 window=5,
                 min_count=5,
                 sg=1,
                 workers=2)

# vector_size, kaç boyutlu word embeddings istiyorsak burda belirtiyoruz.
# Bir token ile diğer tokenler arasındaki anlamsal ilişkiler kurulurken bu tokenin kaç önce ve sonrasındaki tokenler dikkate
# alınarak eğitim yapılmasını istiyorsak window parametresinde belirtiyoruz. Tavsiye edilen 5-15 sayıları arasındadır.
# min_count, corpusta 5 veya daha az geçen tokenler eğitime dahil edilmez. Genellikle 3,4,5 gibi sayılar tercih edilir.
# sg =1, eğitimi skipgram algoritması ile yap.
# sg =0, eğitimi CBOW algoritması ile yap.
# alpha= learning rate
# workers=eğitim için kullanacağımız çekirdek sayısı

CPU times: user 6min 17s, sys: 1.61 s, total: 6min 19s
Wall time: 3min 52s


In [10]:
model.wv['ankara']

# 100 elemanlı/boyutlu word_embedding

array([ 2.74438839e-02,  3.02957267e-01, -1.01784386e-01, -1.56511113e-01,
        2.76780903e-01,  1.93887368e-01, -1.92103952e-01,  6.33531809e-01,
       -1.65956989e-02, -7.59091079e-01, -1.95657820e-01, -7.32019961e-01,
       -2.65674233e-01, -1.40793473e-01,  2.01222271e-01,  6.19625926e-01,
        1.64915040e-01, -1.79271966e-01, -2.50681974e-02, -7.07142770e-01,
        7.52288476e-02,  5.03354609e-01,  6.40448391e-01, -2.62706608e-01,
       -1.04442611e-01,  2.77420580e-01, -4.81181920e-01, -2.35335484e-01,
       -4.18507874e-01,  6.49932086e-01,  4.16625232e-01,  1.86726853e-01,
        1.77415922e-01, -5.55045426e-01, -1.58076346e-01,  8.71169940e-02,
        1.65757418e-01, -4.11534727e-01,  3.26733202e-01, -4.23221141e-01,
        4.34209883e-01, -1.71994697e-02,  1.24352783e-01,  1.39335155e-01,
        1.86631843e-01,  1.39703572e-01,  1.37032852e-01,  2.20282987e-01,
        7.46816576e-01, -3.71209025e-01,  4.25269336e-01, -1.97032303e-01,
        4.03461717e-02,  

In [11]:
model.wv.most_similar('öğretmen')

[('okuldaki', 0.7492612600326538),
 ('erkek', 0.7481780052185059),
 ('öğretmenin', 0.7390701174736023),
 ('öğretmeni', 0.7371501922607422),
 ('üniversite', 0.7220643162727356),
 ('öğrencisinin', 0.7188251614570618),
 ('erkekten', 0.715896487236023),
 ('öğrenci', 0.7132596373558044),
 ('hizmetli', 0.6982784867286682),
 ('öğrencinin', 0.6966313719749451)]

In [12]:
model.wv.most_similar('kırmızı')

[('çizgileri', 0.6546631455421448),
 ('sarı', 0.6499119400978088),
 ('renkli', 0.6316047310829163),
 ('siyah', 0.6177324056625366),
 ('gömlekli', 0.6089180707931519),
 ('çizgi', 0.6053758859634399),
 ('turuncu', 0.5984118580818176),
 ('halı', 0.5958287715911865),
 ('bülten', 0.5914483070373535),
 ('renkte', 0.5867157578468323)]

In [13]:
model.wv.most_similar('eve')

[('evine', 0.8179585337638855),
 ('dükkana', 0.7528007626533508),
 ('mağazaya', 0.7453551888465881),
 ('karakola', 0.7438392043113708),
 ('apartmana', 0.7390093207359314),
 ('köye', 0.7050694227218628),
 ('hapishaneye', 0.704680323600769),
 ('arabaya', 0.7018750905990601),
 ('ofise', 0.7017014622688293),
 ('otobüse', 0.7012637853622437)]

In [14]:
model.wv.most_similar('mavi')

[('marmara', 0.8885944485664368),
 ('gemisine', 0.6669437885284424),
 ('baskınıyla', 0.6455132365226746),
 ('filo', 0.6233791708946228),
 ('filosundaki', 0.6062126159667969),
 ('baskınına', 0.601915180683136),
 ('dökme', 0.6005128026008606),
 ('baskınının', 0.5989163517951965),
 ('filodaki', 0.5973966717720032),
 ('israil', 0.5899045467376709)]

In [15]:
model.wv.most_similar(positive=['öğrenme', 'doktor'], negative=['tedavi'], topn=5)

[('bilişsel', 0.6631882786750793),
 ('olabilmeleri', 0.6611493825912476),
 ('kaliteli', 0.65329509973526),
 ('soy', 0.6532273292541504),
 ('yaptıklarında', 0.6530002951622009)]

In [16]:
model.wv.most_similar(positive=['ankara', 'belçika'], negative=['brüksel'], topn=5)

[('hollanda', 0.6185110211372375),
 ('danimarka', 0.6158261299133301),
 ('avusturya', 0.6007847189903259),
 ('fransa', 0.5896060466766357),
 ('tsk', 0.5743513703346252)]

In [17]:
model.save("word2vec.model")

In [18]:
model = Word2Vec.load("word2vec.model")

## Glove

In [19]:
from gensim.models import KeyedVectors

# KeyedVectors fonksiyonunu farklı bir formattaki word embeddingleri word2vec formatına dönüştürmek için kullanıyoruz.

In [21]:
glove_model = '/content/drive/MyDrive/glove.6B.100d.glove'

model2 = KeyedVectors.load_word2vec_format(glove_model) #'/content/drive/MyDrive/glove.6B.100d.glove'

In [22]:
model2['teacher']

array([ 0.44374 ,  0.67311 , -0.51096 ,  0.20882 , -0.10662 ,  0.55098 ,
       -0.035593,  0.25126 , -0.32789 ,  1.0762  , -0.49637 , -0.4298  ,
        0.36764 ,  0.57894 , -0.25027 , -0.41021 ,  0.086998, -0.16843 ,
       -0.85764 ,  1.0404  , -1.0314  ,  0.095147,  0.30729 ,  0.12348 ,
        0.22745 , -0.52157 , -0.72478 , -1.0843  ,  0.035966,  0.62985 ,
       -1.0991  ,  0.67161 ,  0.33797 ,  0.14551 , -0.90049 , -0.064415,
       -0.75247 ,  0.21741 ,  0.51594 , -0.46291 , -0.77598 ,  0.40705 ,
        0.1889  , -0.43402 ,  0.23202 , -0.081453, -0.3882  , -0.34444 ,
        0.080225, -0.28274 , -0.38869 , -0.58152 , -0.25558 ,  1.0027  ,
       -0.11114 , -1.5402  , -0.16761 , -0.26558 ,  0.9325  ,  0.069397,
        0.96618 ,  0.15449 , -0.22905 , -0.1761  ,  0.13225 , -0.55741 ,
        0.9234  , -0.04845 ,  0.50202 ,  1.0144  , -0.1256  ,  0.30486 ,
        0.090808,  0.17642 , -0.23146 ,  0.68386 ,  0.37269 , -0.37316 ,
       -0.025728, -1.0279  , -0.33142 ,  0.036028, 

In [23]:
model2.most_similar('ankara')

[('turkey', 0.7512096166610718),
 ('istanbul', 0.6787630319595337),
 ('turkish', 0.6690374612808228),
 ('damascus', 0.6372509002685547),
 ('tbilisi', 0.6322181820869446),
 ('erdogan', 0.6258037090301514),
 ('moscow', 0.6217040419578552),
 ('brussels', 0.6181437969207764),
 ('skopje', 0.6164302229881287),
 ('cyprus', 0.606403112411499)]

In [24]:
model2.most_similar('teacher')

[('student', 0.8083398938179016),
 ('school', 0.7545564770698547),
 ('teaching', 0.7521439790725708),
 ('taught', 0.741184651851654),
 ('teachers', 0.7291542887687683),
 ('graduate', 0.7134961485862732),
 ('instructor', 0.7077120542526245),
 ('students', 0.6828974485397339),
 ('teaches', 0.6552315354347229),
 ('education', 0.6528989672660828)]

In [25]:
model2.most_similar('doctor')

[('physician', 0.7673240303993225),
 ('nurse', 0.75215083360672),
 ('dr.', 0.7175194025039673),
 ('doctors', 0.7080884575843811),
 ('patient', 0.7074184417724609),
 ('medical', 0.6995992660522461),
 ('surgeon', 0.6905338168144226),
 ('hospital', 0.6900930404663086),
 ('psychiatrist', 0.658909797668457),
 ('dentist', 0.6447421312332153)]

In [26]:
model2.most_similar(positive=['woman', 'son'], negative=['man'], topn=1)

[('daughter', 0.9090957641601562)]

In [27]:
model2.most_similar(positive=['woman', 'father'], negative=['man'], topn=1)

[('mother', 0.9024619460105896)]

In [28]:
model2.most_similar(positive=['woman', 'uncle'], negative=['man'], topn=1)

[('aunt', 0.8368030190467834)]

In [29]:
model2.most_similar(positive=['ankara', 'germany'], negative=['berlin'], topn=1)

[('turkey', 0.81471186876297)]

In [30]:
model2.most_similar(positive=['teach', 'doctor'], negative=['treat'], topn=1)

[('teacher', 0.7610154151916504)]

In [31]:
model2.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

[('queen', 0.7698540687561035)]

In [32]:
model2.most_similar(positive=['love', 'jealous'], negative=['hate'], topn=1)

[('lover', 0.7032662630081177)]